In [ ]:
from ggv.utils.context import create_default_context
from ggv.utils.macros import google_sheet_to_bigquery
import warnings
import pandas as pd
import numpy as np
import plotly
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import plotly
import seaborn as sns
from apyori import apriori
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
from itertools import combinations
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from sklearn.decomposition import PCA

In [ ]:
df = pd.read_csv('data_recommend.csv')

In [ ]:
 def gcp_cust_profile()->str:
     query=f"""
 WITH
   delivery_detail AS (
   SELECT
     orders.system_order_request_id delivery_order_id,
     orders.user_id delivery_user_id,
     org.organization_name,
     orders.status,
     EXTRACT(DATE FROM orders.completed_at) order_completed_date,
     STRUCT(orders.shipper_name as shipper_name,orders.shipper_phone as shipper_phone) shipper_detail,
     STRUCT(orders.recipient_name as name,orders.recipient_phone as recipient_phone) recipient_detail,
     delivery.courier_id,
     delivery.organization_id,
     LOWER(TRIM(REGEXP_REPLACE(delivery.info, '🚨|🔥', ''))) delivery_info,
     delivery.remark delivery_remark
   FROM
     `TableName` orders
   INNER JOIN
     `TableName` delivery ON orders.system_order_request_id=delivery.id and orders.product_name='DELIVERY'
   LEFT JOIN 
     `TableName` org ON delivery.organization_id=org.organization_id
   WHERE
     orders.status NOT IN ("cancelled",
       "failed","pending","returned")),
 delivery_order_driver_type AS (
 SELECT
   dd.*,
   driver.full_name courier_name,
   driver_type.vehicle courier_type
 FROM
   delivery_detail dd
 LEFT JOIN
   `TableName` driver
 ON
   dd.courier_id=driver.driver_id
 LEFT JOIN
   `TableName` driver_type
 ON
   driver.vehicle_cd = driver_type.cd
 ),
 -- select * from driver_type order by order_completed_date
 categorised_delivery_item as
 (SELECT 
   delivery_order_driver_type.*,
   COALESCE(item_lookup.keyword,'No keyword') keyword ,
   COALESCE(item_lookup.category,'uncategorised') category,
   ROW_NUMBER() OVER (PARTITION BY delivery_order_id ORDER BY LENGTH(item_lookup.keyword) DESC) length_order
 FROM
   delivery_order_driver_type 
 LEFT JOIN
   `TableName` item_lookup
 ON delivery_order_driver_type.delivery_info LIKE CONCAT('%',item_lookup.keyword ,'%')
 ORDER BY
   order_completed_date DESC)
 SELECT 
   delivery_user_id,
   STRING_AGG(category, ',') AS categories
 FROM categorised_delivery_item 
 WHERE length_order=1 AND category <> "uncategorised"
 GROUP BY delivery_user_id
 order by 1 desc
     """
     return big_query_db.df_from_sql(query)

In [ ]:
chunk_size = 10000  
df_chunks = pd.read_csv('data_recommend.csv', chunksize=chunk_size)  

category_counts = {}

for chunk in df_chunks:
    chunk['categories'] = chunk['categories'].str.split(',')
    for categories in chunk['categories']:
        for category in categories:
            if category in category_counts:
                category_counts[category] += 1
            else:
                category_counts[category] = 1

sorted_categories = sorted(category_counts.items(), key=lambda x: x[1], reverse=True)

top_10_categories = [category for category, count in sorted_categories[:10]]

print(sorted_categories)

In [ ]:
df = pd.concat([df, categories_split], axis=1)

In [ ]:
categories_df = df['categories'].str.get_dummies(',')

df = pd.concat([df, categories_df], axis=1)

df.drop('categories', axis=1, inplace=True)
df.drop('delivery_user_id', axis=1, inplace=True)

In [ ]:
transactions = df.values.tolist()

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

In [ ]:
frequent_itemsets = apriori(df_encoded, min_support=0.05, use_colnames=True)

frequent_itemsets = frequent_itemsets.sort_values(by='support', ascending=False)

print(frequent_itemsets)

In [ ]:
df['categories_list'] = df['categories'].str.split(',')

unique_categories = df.groupby('delivery_user_id')['categories_list'].apply(lambda x: len(set([category for categories in x for category in categories])))

print(unique_categories)

In [ ]:
data = list(df_2["categories"].apply(lambda x:x.split(",") ))

In [ ]:
a = TransactionEncoder()
a_data = a.fit(data).transform(data)
a_data

In [ ]:
df_3 = pd.DataFrame(a_data,columns=a.columns_)
df_3 = df_3.replace(False,0)

In [ ]:
df_4 = apriori(df_3, min_support = 0.01, use_colnames = True, verbose = 1)

In [ ]:
df_ar = association_rules(df_4, metric = "confidence", min_threshold = 0.1)